# Ground state of Al-Mg
In this example the ground state of Al-Mg is found using an already trained Cluster Expansion and simulated annealing. The cluster expansion expresses the energy of a structure as

$E = c_0 + c_1\sum_i \sigma_i + c_2 \sum_{\langle ij \rangle} \sigma_i \sigma_j + \cdots + \sum_{\langle ijk \rangle} \sigma_i \sigma_j \sigma_k + \cdots$

The coefficients $c_0, c_1...$ etc. are fitted to total energies from DFT simulations.

## Simulated Annealing
The system starts out at a high temperature, here $T=1000$ K and then it is slowly cooled.
At each temperature $N_{MC}$ Monte Carlo steps are performed as follow

1. Swap two atoms and compute the difference $\Delta E = E_{new} - E_{old}$ between them
2. With probability min$(1,e^{-\Delta E/kT})$ accept the new state, where $k$ is the Boltzmann constant

As the temperature is decreased the system will occupy states with decreasing energy, and finally the system will reach the ground state.

## Setting up the simulation
First we specify the concentration range and initialize the *BulkCrystal* object

In [1]:
from ase.ce.settings import BulkCrystal
conc_args = {
        "conc_ratio_min_1":[[1,0]],
        "conc_ratio_max_1":[[0,1]],
    }
lattice_type = "fcc"
a_lat = 4.05                                     # Lattice parameter in Å
c_lat = None                                     # There are only one lattice parameter for FCC, so c_lat is None
cell_size = [4,4,4]                              # Size of the unit cell
n_site_types = 1                                 # Number of site types. Al and Mg can occupy the same sites, so there are only one
elements = [["Al","Mg"]]                         # Elements that can occupy the different site types
db_name = "database_with_dft_structures.db"      # Data base with DFT calculations
bc = BulkCrystal( lattice_type, a_lat, c_lat, cell_size, n_site_types, elements, conc_args, db_name, max_cluster_size=4 )

Define the different cluster interactions, typically obtained from the *ase.ce.evaluate.Evaluate* object.
Here they are just hard coded into a dictionary.

In [2]:
eci = {"c3_1225_4_1": -0.0017448109612305434, 
 "c2_1000_1_1": -0.02253231472540913, 
 "c4_1225_8_1": 0.0015986520863819958, 
 "c2_707_1_1": 0.0020761708499214765, 
 "c4_707_1_1": -1.5475822532285122e-05, 
 "c4_1225_3_1": 0.0013284466570874605, 
 "c1_1": -1.068187483782512, 
 "c3_1225_2_1": -0.0015608053756936988, 
 "c3_1225_1_1": -0.0010685006728372629, 
 "c0": -2.6460513669066836, 
 "c4_1225_9_1": -7.3952137244461468e-05}

No the Cluster Expansion calculator has to be initialized

In [3]:
from cemc.wanglandau.ce_calculator import CE
# The atoms object in the BulkCrystal object is initially filled with atoms of the first entry in the element 
# list. Here this is Al. Pure Al has a value of 1 in the correlation function
init_cf = {key:1.0 for key in eci.keys()}
calc = CE( bc, eci, initial_cf=init_cf)
bc.atoms.set_calculator( calc )

Now the CE calculator has a reference to the Atoms object of BulkCrystal. **At this point the atoms object should not be modified by other functions that methods of the CE object.** Now we define the Mg concentation we want to find the ground state of. 

In [4]:
mg_conc = 0.25
n_mg_atoms = int( mg_conc*len( bc.atoms ) )

composition = {
    "Al":1.0-mg_conc,
    "Mg":mg_conc
}

bc.atoms._calc.set_composition(composition)

Define temperatures and the number of Monte Carlo steps to perform at each temperature. It is wise to cool slowly towards the end

In [5]:
temps = [800,700,500,300,200,100,50,20,19,18,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
n_steps_per = 100 # Consider more steps per temperature than this

To be sure that we store the structure with the lowest known energy we need keep track of which structure this is as the simulation proceeds. The *cemc* modules provides a special object that can monitor any Monte Carlo simulation, known as *MCObservers*. Here we will use the observer *LowestEnergyStructure* which essentially always stores the structure with the lowest energy.

In [6]:
from cemc.mcmc.mc_observers import LowestEnergyStructure
obs = LowestEnergyStructure( calc, None )

Import the Montecarlo object and loop over the temperatures

In [7]:
from cemc.mcmc.montecarlo import Montecarlo
for T in temps:
    print ("Current temperature {}".format(T))
    mc_obj = Montecarlo( bc.atoms, T )
    obs.mc_obj = mc_obj # Set the reference to the Montecarlo object
    mc_obj.attach( obs ) # Attach the observer to the Montecarlo object
    mc_obj.runMC( steps=n_steps_per, verbose=False )

Current temperature 800
Current temperature 700
Current temperature 500
Current temperature 300
Current temperature 200
Current temperature 100
Current temperature 50
Current temperature 20
Current temperature 19
Current temperature 18
Current temperature 15
Current temperature 14
Current temperature 13
Current temperature 12
Current temperature 11
Current temperature 10
Current temperature 9
Current temperature 8
Current temperature 7
Current temperature 6
Current temperature 5
Current temperature 4
Current temperature 3
Current temperature 2
Current temperature 1


Now one can save the resulting ground state structure.

In [8]:
from ase.io import write
write( "ground_state.xyz", bc.atoms )

Examples of how the structures might look like is shown below
![Structure examples](../img/gs_all_states.png)